# BAPETCO PDF REPORT READER

reads an Asset pdf oil and gas production report and converts it to a desired format to be loaded into the system database.


In [1]:
import io
import re
import camelot
import pandas as pd
from datetime import datetime
from tkinter import Tk 
from tkinter.filedialog import askopenfilename, askdirectory
import time
import shutil
import os

start_time = time.time()

def extracted_values_to_df_r(df,col_name):
    for k in range(0,len(df_r)):
        for i in range (0,len(df)):
            if df_r['Date'][k] == df['Date'][i]:
                df_r[col_name][k]=df['Value'][i]
            else:
                continue

def textract(filepath, pdfpage):
    tables = camelot.read_pdf(filepath, pages=str(pdfpage))
    df={}
    x=1
    while x<len(tables):
        df[x] = tables[x-1].df
        x+=1
    return df
                                                                  #creates a backup for the main excel file in Egypt folder                
now = datetime.now()
current_time = now.strftime("_%H_%M_%d-%m-%Y")

file_path = r"path_name"
dest1 = r"path_name"

shutil.copy2(file_path, dest1)

os.rename(r"path_name".format(current_time))

                                                                  #reading Egypt sheet from main excel file in NOV location
df_r = pd.read_excel(r"path_name", sheet_name="Egypt", index_col=None, header=1)

                                                                 #messagebox asking for pdf file
Tk().withdraw()                                                  #* save and then select pdf report from a specially dedicated folder "Egypt"
path = askopenfilename(title='SELECT BAPETCO PDF REPORT')




pages = 2                                                        #reading page 2 and extrcting 3 tables (oil,cond,gas)
df_t = textract(path, pages)
                                                                 
for x, value in df_t.items(): 
    if len(df_t[x].columns)>8:
        df_t[x].columns = df_t[x].iloc[0]
        df_t[x].drop([0], inplace=True)
        df_t[x].set_index('Field', inplace=True)
            
    
                                                                 #reading page 5 and extracting water injection table
pages = 5
df_w = textract(path, pages)
                                                                 #cleaning water injection table
dates_head = str(list(df_w[3].iloc[0]))
bad_chars='\'[],'
for c in bad_chars: 
    dates_head = dates_head.replace(c, "")
dates_head = re.split(' +', dates_head)
    
df_w[3].columns =  dates_head
df_w[3].drop(columns=['diff','Average:'],inplace=True)
df_w[3].drop([0], axis=0, inplace=True)
    
                                                                #creating dataframes
df_t[1].drop(df_t[1].columns[-1],axis=1,inplace=True)
oil=df_t[1].iloc[-1]
cond=df_t[3].iloc[-1]
gas=df_t[4].iloc[-1]
water=df_w[3].iloc[-1]

df_oil=pd.DataFrame({'Date':oil.index[1:], 'Value':oil.values[1:]})
df_cond=pd.DataFrame({'Date':cond.index[1:], 'Value':cond.values[1:]})
df_gas=pd.DataFrame({'Date':gas.index[1:-1], 'Value':gas.values[1:-1]})
df_water=pd.DataFrame({'Date':water.index[1:], 'Value':water.values[1:]})

                                                                #changing date and value format in each dataframe
dataframes=[df_oil,df_gas,df_cond,df_water]
for df in dataframes:
    for i in range(0,len(df)):
        new_date=datetime.strptime(df['Date'][i],'%d-%b-%Y').strftime('%Y-%m-%d')
        df['Date'][i]=new_date
        new_value=df['Value'][i].replace(',','').split('.',1)[0]
        df['Value'][i]=int(new_value)
                                                               #changing date column format in Egypt sheet in main excel file
df_r['Date']=df_r['Date'][2:].dt.strftime('%Y-%m-%d')

                                                               #using 'extracted_values_to_df_r' function defined above
extracted_values_to_df_r(df_gas,'Bapetco Daily Gas')
extracted_values_to_df_r(df_oil,'Bapetco Daily Oil')
extracted_values_to_df_r(df_cond,'Bapetco Daily Condensate')
extracted_values_to_df_r(df_water,'BAPETCO daily water injection')
                                                                #brings back 'Date' column datetime format for excel vlookup to work properly
df_r['Date']=pd.to_datetime(df_r['Date'])
                                                                #saving the outome with newly populated rows in separate folder in NOV location
out_path = r'path_name'
writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
df_r.to_excel(writer, sheet_name='Egypt')
writer.save()

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Filip.Mazurkiewicz\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


--- 58.19289231300354 seconds ---


In [5]:
df_gas

,Date,Value
0,2020-09-29,370
1,2020-09-30,370
2,2020-10-01,370
3,2020-10-02,370
4,2020-10-03,370
5,2020-10-04,371
6,2020-10-05,369


In [21]:
df_r['Date'][2:].dt.strftime('%Y-%m-%d')

2      2020-01-01
3      2020-01-02
4      2020-01-03
5      2020-01-04
6      2020-01-05
          ...    
363    2020-12-27
364    2020-12-28
365    2020-12-29
366    2020-12-30
367    2020-12-31
Name: Date, Length: 366, dtype: object

In [20]:
df['Date']

0    2020-09-01
1    2020-09-02
2    2020-09-03
3    2020-09-04
4    2020-09-05
5    2020-09-06
6    2020-09-07
Name: Date, dtype: object

In [23]:
df_cond

,Date,Value
0,2020-09-01,8033
1,2020-09-02,8364
2,2020-09-03,8104
3,2020-09-04,8166
4,2020-09-05,8240
5,2020-09-06,8132
6,2020-09-07,8320


In [6]:
df_r[250:290]

,Date,Bapetco Daily Condensate,Bapetco Daily Condensate.1,Bapetco Daily Gas,Bapetco Daily Gas.1,Bapetco Daily Oil,Bapetco Daily Oil.1,BAPETCO daily water injection,Unnamed: 8,Unnamed: 9,...,GAS,OIL,WAT,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
250,2020-09-05,8166,1298.4,372,1.05136e+07,30585,6161.42,67011,NaN,NaN,...,372,30456,67284,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,2020-09-06,8240,1310.16,374,1.05701e+07,32040,6404.53,66068,NaN,NaN,...,372,30585,67011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,2020-09-07,8132,1292.99,377,1.06549e+07,32119,6399.92,64824,NaN,NaN,...,374,32040,66068,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,2020-09-08,8320,1322.88,378,1.06831e+07,32090,6425.2,63707,NaN,NaN,...,377,32119,64824,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,2020-09-09,7968,1266.91,376,1.06266e+07,32214,6388.95,52609,NaN,NaN,...,378,32090,63707,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,2020-09-10,8289,1317.95,376,1.06266e+07,32083,6419.16,53539,NaN,NaN,...,376,32214,52609,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,2020-09-11,8106,1288.86,373,1.05418e+07,32216,6411.21,61243,NaN,NaN,...,376,32083,53539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,2020-09-12,8093,1286.79,374,1.05701e+07,32139,6396.9,61242,NaN,NaN,...,373,32216,61243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,2020-09-13,7829,1244.81,365,1.03157e+07,32105,6349.51,62021,NaN,NaN,...,374,32139,61242,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,2020-09-14,7914,1258.33,362,1.02309e+07,32195,6377.34,61998,NaN,NaN,...,365,32105,62021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
